In [1]:
import requests
from bs4 import BeautifulSoup as BS
import pandas as pd
import time
import matplotlib.pyplot as plt
import numpy as np
import re
import ast
pd.set_option('display.max_columns', 100)
pd.options.display.max_rows = 4000

In [2]:
def get_hrefs(year):
    page = requests.get('https://www.basketball-reference.com/draft/NBA_'+str(year)+'.html')
    #print('here')
    soup = BS(page.content, 'html.parser')
    table_body=soup.find('tbody')
    rows = table_body.find_all('tr')
    hrefs=[]
    for row in rows:
        try:
            href=row.find_all('a')
            x= (href[2])
            x= re.findall('\"+(/players/+.*)\"', str(x))
            if len(x)>0:
                hrefs.append(x)
            else:
                pass
            continue
        except:
            pass
    return hrefs

In [6]:
def get_player_info(href1):
    page = requests.get('https://www.basketball-reference.com'+str(href1))
    #Name of Player
    soup = BS(page.content, 'html.parser')
    table_body=soup.find_all(itemprop= "name")
    table_body
    name= re.findall('<h1 itemprop=\"name\">(.*)</h1>', str(table_body))
    if len(name)==0:
        name= re.findall('<span>(.*)</span>', str(table_body))
    else:
        pass
    print(name)
    #Bio
    soup = BS(page.content, 'html.parser')
    table_body=soup.find_all('p')
    work=table_body[0:10]
    #print(work)
    height= re.findall('(\d*cm)', str(work))
    weight= re.findall('(\d*kg)', str(work))
    bday= re.findall('data-birth=\"(\d*\-\d*\-\d*)', str(work))
    #accolades
    accolades_body=soup.find(id="bling")
    All_Star_apps= re.findall('(\d*)x All Star', str(accolades_body))
    All_NBA_apps= re.findall('(\d*)x All-NBA', str(accolades_body))
    All_Def_apps= re.findall('(\d*)x All-Defensive', str(accolades_body))
    HOF= re.findall('(Hall of Fame)', str(accolades_body))
    empty_list=0
    if len(All_Star_apps)==empty_list:
        All_Star_apps='0'
    else:
        All_Star_apps=All_Star_apps[0]
    if len(All_NBA_apps)==empty_list:
        All_NBA_apps='0'
    else:
        All_NBA_apps=All_NBA_apps[0]
    if len(All_Def_apps)==empty_list:
        All_Def_apps='0'
    else:
        All_Def_apps= All_Def_apps[0]
    if len(HOF)==empty_list:
        HOF='0'
    else:
        HOF= HOF[0]
    bio= [name[0],height[0],weight[0],bday[0],All_Star_apps,All_NBA_apps,All_Def_apps,HOF]
    return bio

In [15]:
def rookie_data(year):
    hrefs= get_hrefs(year)
    players= []
    for href in range(len(hrefs)):
        player= get_player_info(hrefs[href][0])
        players.append(player)
        time.sleep(.3)
    return players

In [ ]:
Draft_years= []
for i in range(1986,2001):
    Draft_years.append(str(i))
df_rookie_data = pd.DataFrame(rookie_data(1985), columns=['Name','Height','Weight','Birthday','All_Star_apps','All_NBA_apps','All_Def_apps','HOF'])
for year in Draft_years:
    print(year)
    df_rookie_data2= pd.DataFrame(rookie_data(year), columns=['Name','Height','Weight','Birthday','All_Star_apps','All_NBA_apps','All_Def_apps','HOF'])
    df_rookie_data=pd.concat([df_rookie_data,df_rookie_data2])
    df_rookie_data.drop_duplicates(subset ='Name', keep = 'first', inplace = True)
    time.sleep(1)

['Patrick Ewing']
['Wayman Tisdale']
['Benoit Benjamin']
['Xavier McDaniel']
['Jon Koncak']
['Joe Kleine']
['Chris Mullin']
['Detlef Schrempf']
['Charles Oakley']
['Ed Pinckney']
['Keith Lee']
['Kenny Green']
['Karl Malone']
['Alfredrick Hughes']
['Blair Rasmussen']
['Bill Wennington']
['Uwe Blab']
['Joe Dumars']
['Steve Harris']
['Sam Vincent']
['Terry Catledge']
['Jerry Reynolds']
['A.C. Green']
['Terry Porter']
['Mike Smrek']
['Bill Martin']
['Dwayne McClain']
['Ken Johnson']
['Mike Brittain']
['Manute Bol']
['Nick Vanos']
['Greg Stokes']
['Tyrone Corbin']
['Yvon Joseph']
['Carey Scurry']
['Fernando Martín']
['Mark Acres']
['Lorenzo Charles']
['Barry Stevens']
['Voise Winters']
['Hot Rod Williams']
['Adrian Branch']
['Gerald Wilkins']
['Brad Wright']
['Sam Mitchell']
['Sedric Toney']
['Perry Young']
['Harold Keeling']
['Michael Adams']
['Mike Brown']
['Fred Cofield']
['Alex Stivrins']
['Arvydas Sabonis']
['Mark Davis']
['Scott Roth']
['Delaney Rudd']
['John Battle']
['Spud Webb']
['

In [18]:
df_info

116

In [ ]:
df_info['name']= df_info.name.map(lambda x: x.lower())